In [1]:
import pandas as pd 

In [4]:
customers = pd.read_csv('./data/recommend_1.csv')

In [5]:
transactions = pd.read_csv('./data/trx_data.csv')

In [8]:
print(customers.shape)

(1000, 1)


In [7]:
customers.head()

,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [9]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [10]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [11]:
#organising the data
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [14]:
import time
s = time.time()
import numpy as np

In [15]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.23 minutes


In [17]:
data

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2
...,...,...,...
133580,28596,211,3
133581,28596,255,1
133582,28598,212,1
133583,28604,282,1


In [20]:
data.shape

(133585, 3)

In [21]:
#creating dummy - normalising the data - customer purchsed or not

def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [22]:
data_dummy = create_data_dummy(data)

In [23]:
#normalising the data for every one
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
(df_matrix.shape)

(24429, 300)

In [26]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#making a table for the inout data

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [29]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
#splitting the data for test and train
train, test = train_test_split(data, test_size = .2)

In [34]:
train.shape

(106868, 3)

In [35]:
test.shape

(26717, 3)

In [38]:
import turicreate as tc

In [39]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [40]:
train_data

customerId,productId,purchase_count
5336,1,2
26994,159,1
19726,4,1
2721,250,1
14503,125,3
2946,121,2
24559,272,1
33,141,1
2182,65,1
157,32,1


In [41]:
test_data

customerId,productId,purchase_count
1979,99,1
3087,176,1
2252,48,2
9341,46,2
7624,16,3
144,4,1
10960,148,1
3973,96,2
24793,106,1
15367,188,1


In [42]:
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [43]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [57]:
#Using a Popularity model as a baseline

In [45]:
#using popularity model
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [46]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.143785s

106868 observations to process; with 300 unique items.

In [47]:
#printing the recommendations
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 51413.9

recommendations finished on 2000/62483 queries. users per second: 91886.4

recommendations finished on 3000/62483 queries. users per second: 117000

recommendations finished on 4000/62483 queries. users per second: 129803

recommendations finished on 5000/62483 queries. users per second: 152472

...

recommendations finished on 59000/62483 queries. users per second: 193902

recommendations finished on 60000/62483 queries. users per second: 194710

recommendations finished on 61000/62483 queries. users per second: 195281

recommendations finished on 62000/62483 queries. users per second: 194484

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    248    | 3.2444444444444445 |  1   |
|     0      |    132    | 3.076923076923077  |  2   |
|     0      |     34    | 3.0274509803921568 |  3   |
|     0      |     37    | 3.024390243902439  |  4   |
|     0      |     0     | 2.9495238095238094 |  5   |
|     0      |     27    | 2.9015151515151514 |  6   |
|     0      |    110    | 2.7942857142857145 |  7   |
|     0      |     3     | 2.7494736842105265 |  8   |
|     0      |    230    | 2.6766917293233083 |  9   |
|     0      |     32    | 2.6018518518518516 |  10  |
|     1      |    248    | 3.2444444444444445 |  1   |
|     1      |    132    | 3.076923076923077  |  2   |
|     1      |     34    | 3.0274509803921568 |  3   |
|     1      |     37    | 3.024390243902439  |  4   |
|     1      |     0     | 2.9495238095238094 |  5   |
|     1   

In [48]:
#model for best fit model
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [49]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [50]:
#using the purchase dummy
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23293 users and 300 items.

Data prepared in: 0.127641s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 140489

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    255    |  1.0  |  1   |
|    1553    |     35    |  1.0  |  2   |
|    1553    |     54    |  1.0  |  3   |
|    1553    |     24    |  1.0  |  4   |
|    1553    |     41    |  1.0  |  5   |
|    1553    |     84    |  1.0  |  6   |
|    1553    |    143    |  1.0  |  7   |
|    1553    |     68    |  1.0  |  8   |
|    1553    |    188    |  1.0  |  9   |
|    1553    |    267    |  1.0  |  10  |
|   20400    |    255    |  1.0  |  1   |
|   20400    |     35    |  1.0  |  2   |
|   20400    |     54    |  1.0  |  3   |
|   20400    |     24    |  1.0  |  4   |
|   20400    |     41    |  1.0  |  5   |
|   20400    |     84    |  1.0  |  6   |
|   20400    |    143    |  1.0  |  7   |
|   20400    |     68    |  1.0  |  8   |
|   20400    |    188    |  1.0  |  9   |
|   20400    |    267    |  1.0  |  10  |
|   19750    |    255    |  1.0  |

In [51]:
#using normalized purchase count
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23344 users and 300 items.

Data prepared in: 0.134043s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 107527

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7386363636363636 |  1   |
|    1553    |    247    | 0.32492997198879564 |  2   |
|    1553    |    230    |  0.3208955223880592 |  3   |
|    1553    |    125    | 0.26056338028168974 |  4   |
|    1553    |    294    |  0.2543999999999996 |  5   |
|    1553    |    204    | 0.24395604395604376 |  6   |
|    1553    |    248    | 0.24390243902439024 |  7   |
|    1553    |    155    | 0.23076923076923073 |  8   |
|    1553    |    276    |  0.2246376811594203 |  9   |
|    1553    |     72    |  0.2231237322515213 |  10  |
|   20400    |    226    |  0.7386363636363636 |  1   |
|   20400    |    247    | 0.32492997198879564 |  2   |
|   20400    |    230    |  0.3208955223880592 |  3   |
|   20400    |    125    | 0.26056338028168974 |  4   |
|   20400    |    294    |  0.2543999999999996 |

In [52]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
248    3.244444
132    3.076923
34     3.027451
37     3.024390
0      2.949524
27     2.901515
110    2.794286
3      2.749474
230    2.676692
32     2.601852
10     2.588235
245    2.533333
226    2.500000
68     2.455696
58     2.444444
252    2.431034
129    2.427711
82     2.423868
18     2.392500
87     2.383065
Name: purchase_count, dtype: float64

In [58]:
#Cosine similarity

In [53]:
#using the purchase count
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.137986s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.04323s

recommendations finished on 1000/1000 queries. users per second: 52034.6

In [54]:
#using the purchase dummy
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23293 users and 300 items.

Data prepared in: 0.124008s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.026215s

recommendations finished on 1000/1000 queries. users per second: 78070.1

In [56]:
#using normalized purchase count
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23344 users and 300 items.

Data prepared in: 0.122754s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.073367s

recommendations finished on 1000/1000 queries. users per second: 34174

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     20    |          0.0          |  1   |
|    1553    |    108    |          0.0          |  2   |
|    1553    |    160    |          0.0          |  3   |
|    1553    |     39    |          0.0          |  4   |
|    1553    |    240    |          0.0          |  5   |
|    1553    |    137    |          0.0          |  6   |
|    1553    |    230    |          0.0          |  7   |
|    1553    |     5     |          0.0          |  8   |
|    1553    |    218    |          0.0          |  9   |
|    1553    |    136    |          0.0          |  10  |
|   20400    |     20    |          0.0          |  1   |
|   20400    |    108    |          0.0          |  2   |
|   20400    |    160    |          0.0          |  3   |
|   20400    |     39    |          0.0          |  4   |
|   20400    |

In [59]:
#Pearson similarity

In [60]:
#using the purchase count
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23296 users and 300 items.

Data prepared in: 0.142144s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.098869s

recommendations finished on 1000/1000 queries. users per second: 81646

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.2394354340102933 |  1   |
|    1553    |    132    | 3.0769230769230775 |  2   |
|    1553    |     37    | 3.0243902439024417 |  3   |
|    1553    |     34    | 3.0045389102370126 |  4   |
|    1553    |     0     | 2.949523809523812  |  5   |
|    1553    |     27    | 2.9015151515151514 |  6   |
|    1553    |    110    | 2.789272069590432  |  7   |
|    1553    |     3     | 2.7488462559172997 |  8   |
|    1553    |    230    | 2.6755180393618736 |  9   |
|    1553    |     32    | 2.6018518518518503 |  10  |
|   20400    |    248    | 3.244444444444444  |  1   |
|   20400    |    132    | 3.0651799486233644 |  2   |
|   20400    |     34    | 3.0274509803921563 |  3   |
|   20400    |     37    | 3.0243902439024417 |  4   |
|   20400    |     0     | 2.949523809523812  |  5   |
|   20400 

In [62]:
#using the purchase dummy
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23293 users and 300 items.

Data prepared in: 0.139554s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.103993s

recommendations finished on 1000/1000 queries. users per second: 98658.2

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    255    |  0.0  |  1   |
|    1553    |     35    |  0.0  |  2   |
|    1553    |     54    |  0.0  |  3   |
|    1553    |     24    |  0.0  |  4   |
|    1553    |     41    |  0.0  |  5   |
|    1553    |     84    |  0.0  |  6   |
|    1553    |    143    |  0.0  |  7   |
|    1553    |     68    |  0.0  |  8   |
|    1553    |    188    |  0.0  |  9   |
|    1553    |    267    |  0.0  |  10  |
|   20400    |    255    |  0.0  |  1   |
|   20400    |     35    |  0.0  |  2   |
|   20400    |     54    |  0.0  |  3   |
|   20400    |     24    |  0.0  |  4   |
|   20400    |     41    |  0.0  |  5   |
|   20400    |     84    |  0.0  |  6   |
|   20400    |    143    |  0.0  |  7   |
|   20400    |     68    |  0.0  |  8   |
|   20400    |    188    |  0.0  |  9   |
|   20400    |    267    |  0.0  |  10  |
|   19750    |    255    |  0.0  |

In [64]:
#using the normalized purchase count
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23344 users and 300 items.

Data prepared in: 0.11883s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.087351s

recommendations finished on 1000/1000 queries. users per second: 68045.7

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7382516305554997 |  1   |
|    1553    |    247    | 0.32476100772201844 |  2   |
|    1553    |    230    |  0.320735424312193  |  3   |
|    1553    |    125    |  0.2605633802816901 |  4   |
|    1553    |    294    | 0.25439999999999996 |  5   |
|    1553    |    204    | 0.24395604395604387 |  6   |
|    1553    |    248    |  0.2437650205158606 |  7   |
|    1553    |    155    | 0.23063202087695778 |  8   |
|    1553    |    276    | 0.22430730751459146 |  9   |
|    1553    |     72    | 0.22312373225152113 |  10  |
|   20400    |    226    |  0.7386363636363636 |  1   |
|   20400    |    247    |  0.3249299719887955 |  2   |
|   20400    |    230    |  0.3208955223880597 |  3   |
|   20400    |    125    |  0.2599440880224738 |  4   |
|   20400    |    294    | 0.25439999999999996 |

In [65]:
#Evaluating the models

In [66]:
#variables declaration
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [68]:
#Modles on purchase count
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13915 queries. users per second: 65432.2

recommendations finished on 2000/13915 queries. users per second: 57364.1

recommendations finished on 3000/13915 queries. users per second: 70764.7

recommendations finished on 4000/13915 queries. users per second: 73873

...

recommendations finished on 10000/13915 queries. users per second: 64106.3

recommendations finished on 11000/13915 queries. users per second: 65237.8

recommendations finished on 12000/13915 queries. users per second: 63308.5

recommendations finished on 13000/13915 queries. users per second: 64126


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0010061085159899375 | 0.0005601469237832882 |
|   2    |  0.001185770750988141 | 0.0013225421921074082 |
|   3    |  0.00261109114864056  |  0.004033621985472504 |
|   4    | 0.0032518864534674804 | 0.0068406569768832215 |
|   5    | 0.0061947538627380935 |  0.017101162830274463 |
|   6    |  0.005569529284944301 |  0.01851479092374439  |
|   7    |  0.005235870848519086 |  0.020037704469654773 |
|   8    |  0.005659360402443402 |  0.02489229472309512  |
|   9    |  0.005341957120613303 |  0.026975324082444994 |
|   10   | 0.0051958318361480646 |  0.029561685898055818 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.132630451492796

Per User RMSE (best)
+------------+-----------------------+-------

In [69]:
#model on purchase dummy
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13930 queries. users per second: 84160.9

recommendations finished on 2000/13930 queries. users per second: 107695

recommendations finished on 3000/13930 queries. users per second: 111620

recommendations finished on 4000/13930 queries. users per second: 125196

...

recommendations finished on 10000/13930 queries. users per second: 79862.6

recommendations finished on 11000/13930 queries. users per second: 81423

recommendations finished on 12000/13930 queries. users per second: 76825.6

recommendations finished on 13000/13930 queries. users per second: 77363.9


Precision and recall summary statistics by cutoff
+--------+-----------------------+----------------------+
| cutoff |     mean_precision    |     mean_recall      |
+--------+-----------------------+----------------------+
|   1    | 0.0031586503948312994 | 0.001747228204514636 |
|   2    |  0.004809763101220394 | 0.004584945444600867 |
|   3    |  0.004570471404642259 | 0.006895252390586206 |
|   4    |  0.004558506819813363 |  0.0088011252925108  |
|   5    |  0.005010768126346045 | 0.012453086914680606 |
|   6    |  0.005049054797798514 | 0.01496171671512662  |
|   7    | 0.0062557686391139285 | 0.022218351362285328 |
|   8    |  0.006146805455850681 | 0.02514326504635186  |
|   9    |  0.006516710536811118 | 0.02955630879366095  |
|   10   |  0.006697774587221771 | 0.03376633220245074  |
+--------+-----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------

In [70]:
#model on normalised purchase frequency
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13859 queries. users per second: 72648

recommendations finished on 2000/13859 queries. users per second: 95406.2

recommendations finished on 3000/13859 queries. users per second: 101228

recommendations finished on 4000/13859 queries. users per second: 110439

...

recommendations finished on 10000/13859 queries. users per second: 71927.4

recommendations finished on 11000/13859 queries. users per second: 72133.5

recommendations finished on 12000/13859 queries. users per second: 70203.4

recommendations finished on 13000/13859 queries. users per second: 70212.6


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.003535608629771269 |  0.002126261428880663 |
|   2    |  0.003138754599898981 | 0.0035214951406595863 |
|   3    |  0.002910262885730093 |  0.005142583723289406 |
|   4    | 0.0026877841114077506 |  0.006382652350976183 |
|   5    | 0.0026841763474998183 |  0.00819383276985832  |
|   6    | 0.0024172018183130114 |  0.008876902736426379 |
|   7    | 0.0022883531073155123 |  0.009681655206256813 |
|   8    | 0.0021646583447579217 |  0.01032206867014874  |
|   9    | 0.0022047446104015798 |  0.011493289136592545 |
|   10   |  0.002763547153474295 |  0.015898089274893452 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1315510225807246

Per User RMSE (best)
+------------+-----------------------+------

In [74]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Normalized Purchase Counts']

SyntaxError: invalid syntax (<ipython-input-74-2bb8a738cd98>, line 3)

In [75]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

SyntaxError: invalid syntax (<ipython-input-75-e3cbc62f58c0>, line 1)

In [71]:
#Model Selection

In [76]:
#final output

In [77]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)


Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.123046s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.078564s

recommendations finished on 1000/1000 queries. users per second: 55872.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     1     | 0.10348175764083863  |  1   |
|    1553    |     2     |  0.0934672474861145  |  2   |
|    1553    |     35    |  0.0845762014389038  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.06512556076049805  |  5   |
|    1553    |     15    | 0.06476415395736694  |  6   |
|    1553    |     11    | 0.05467898845672607  |  7   |
|    1553    |     5     | 0.05406981706619263  |  8   |
|    1553    |     36    | 0.05048650503158569  |  9   |
|    1553    |     13    | 0.04985467195510864  |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

In [78]:
#csv file output
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,1,0.103482,1
1,1553,2,0.093467,2
2,1553,35,0.084576,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [79]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [82]:
#defing function to get the desired output
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('./output/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [83]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 48586.1

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|125|294|204|248|155|276|72
11,226|247|230|125|294|204|248|155|276|83
12,226|247|230|125|294|204|248|155|276|72
16,226|247|230|125|294|204|248|155|276|72
21,226|247|230|125|294|204|248|155|276|72


In [84]:
#recommending the customers
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [87]:
customer_recomendation(11)

recommendedProducts    226|247|230|125|294|204|248|155|276|83
Name: 11, dtype: object

In [88]:
customer_recomendation(16)

recommendedProducts    226|247|230|125|294|204|248|155|276|72
Name: 16, dtype: object

In [89]:
customer_recomendation(4)

recommendedProducts    226|247|230|125|294|204|248|155|276|72
Name: 4, dtype: object